In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [21]:
eva_df = pd.read_csv('./stock/return/6757_return.csv')     # 包含 ['日期', '對數報酬率']
taiex_df = pd.read_csv('./stock/return/taiex_return.csv') # 包含 ['日期', '對數報酬率']
eva_feature = pd.read_csv('./output/tiger_feature_1.csv')

In [22]:
eva_feature

,Unnamed: 0,Event Date,Event,Label,Count,Avg Sentiment,Avg Positive Sentiment,Avg Negative Sentiment,Positive Count,Negative Count
0,2,2023/8/16,Event_1,Label_1,4,-0.104365,0.858753,-0.425404,1,3
1,15,2023/11/13,Event_3,Label_3,3,-0.114446,0.016033,-0.179685,1,2
2,8,2023/12/10,Event_2,Label_2,6,-0.290922,0.000000,-0.290922,0,6
3,20,2023/12/10,Event_4,Label_4,5,-0.112486,0.056978,-0.154852,1,4
4,27,2023/12/10,Event_5,Label_5,4,-0.309070,0.276540,-0.504274,1,3
5,9,2024/1/2,Event_2,Label_2,5,-0.514286,0.000000,-0.514286,0,5
6,10,2024/4/29,Event_2,Label_2,4,-0.100835,0.026291,-0.227962,2,2
7,21,2024/5/7,Event_4,Label_4,5,-0.261078,0.056261,-0.340412,1,4
8,28,2024/7/15,Event_5,Label_5,4,-0.259450,0.233168,-0.423656,1,3
9,3,2024/7/26,Event_1,Label_1,3,-0.013849,0.413278,-0.227413,1,2


In [23]:
# 確保欄位型別為 datetime
eva_df['Date'] = pd.to_datetime(eva_df['Date'])
taiex_df['Date'] = pd.to_datetime(taiex_df['Date'])
eva_feature['Event Date'] = pd.to_datetime(eva_feature['Event Date'], errors='coerce')

In [24]:
# 合併 EVA 報酬與 TAIEX 報酬
df = pd.merge(eva_df, taiex_df, on='Date', suffixes=('_eva', '_taiex'))

# 初始化異常報酬欄位
eva_feature['AR'] = np.nan
eva_feature['CAR'] = np.nan

# 將交易日列表先建好，加速查詢
trading_days = df['Date'].sort_values().reset_index(drop=True)

# 處理每筆事件
for i, row in eva_feature.iterrows():
    event_date = row['Event Date']
    if pd.isna(event_date):
        continue

    # 如果事件日不是交易日，調整為往後最近的交易日
    if event_date not in trading_days.values:
        future_trading_days = trading_days[trading_days > event_date]
        if future_trading_days.empty:
            continue  # 如果未來也沒有交易日，跳過
        event_date = future_trading_days.iloc[0]

    # 估計期 [-90, -2] 天內的交易日
    estimation_window = df[(df['Date'] >= event_date - pd.Timedelta(days=90)) &
                           (df['Date'] < event_date - pd.Timedelta(days=2))]

    if len(estimation_window) < 30:
        continue  # 避免回歸不穩定

    # 回歸市場模型：R_i = alpha + beta * R_m
    X = sm.add_constant(estimation_window['Return_taiex'])
    y = estimation_window['Return_eva']
    model = sm.OLS(y, X).fit()
    alpha, beta = model.params['const'], model.params['Return_taiex']

    # 事件視窗 [-1, 0, +1]
    window = df[(df['Date'] >= event_date - pd.Timedelta(days=1)) &
                (df['Date'] <= event_date + pd.Timedelta(days=1))].copy()

    if window.empty or len(window) < 1:
        continue

    # 計算每一天的預期報酬與異常報酬
    window['預期報酬'] = alpha + beta * window['Return_taiex']
    window['異常報酬'] = window['Return_eva'] - window['預期報酬']

    # AR = 當天異常報酬
    ar_row = window[window['Date'] == event_date]
    if not ar_row.empty:
        eva_feature.loc[i, 'AR'] = ar_row['異常報酬'].values[0]

    # CAR = [-1, +1] 異常報酬加總
    eva_feature.loc[i, 'CAR'] = window['異常報酬'].sum()


In [25]:
eva_feature

,Unnamed: 0,Event Date,Event,Label,Count,Avg Sentiment,Avg Positive Sentiment,Avg Negative Sentiment,Positive Count,Negative Count,AR,CAR
0,2,2023-08-16,Event_1,Label_1,4,-0.104365,0.858753,-0.425404,1,3,NaN,NaN
1,15,2023-11-13,Event_3,Label_3,3,-0.114446,0.016033,-0.179685,1,2,NaN,0.000000
2,8,2023-12-10,Event_2,Label_2,6,-0.290922,0.000000,-0.290922,0,6,-0.014048,-0.011394
3,20,2023-12-10,Event_4,Label_4,5,-0.112486,0.056978,-0.154852,1,4,-0.014048,-0.011394
4,27,2023-12-10,Event_5,Label_5,4,-0.309070,0.276540,-0.504274,1,3,-0.014048,-0.011394
5,9,2024-01-02,Event_2,Label_2,5,-0.514286,0.000000,-0.514286,0,5,0.009927,0.010254
6,10,2024-04-29,Event_2,Label_2,4,-0.100835,0.026291,-0.227962,2,2,0.010897,0.002310
7,21,2024-05-07,Event_4,Label_4,5,-0.261078,0.056261,-0.340412,1,4,-0.006335,0.126431
8,28,2024-07-15,Event_5,Label_5,4,-0.259450,0.233168,-0.423656,1,3,-0.059825,-0.103920
9,3,2024-07-26,Event_1,Label_1,3,-0.013849,0.413278,-0.227413,1,2,-0.066080,-0.066080


In [26]:
eva_feature.to_csv("./final/tiger_xy.csv")